# Initialize

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import config
import csv
from datetime import datetime
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

/Users/ryanhermes/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
ticker = 'CRM'

# Data

### Daily Info

In [2]:
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={config.alphavantage_apikey}'
r = requests.get(url)
data = r.json()

header = list(data['Time Series (Daily)']['2024-05-14'].keys())
header.insert(0, 'Date')

# Writing to CSV
with open(f'data/{ticker}_daily.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=header)
    
    writer.writeheader()
    
    for date, values in data['Time Series (Daily)'].items():
        row = {'Date': date}
        row.update(values)
        writer.writerow(row)


### Insider Trades

In [7]:
url = f"http://openinsider.com/screener?s={ticker}&o=&pl=&ph=&ll=&lh=&fd=730&fdr=&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page=1"

# Request the webpage
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

time.sleep(1)

# Find the table
table = soup.find('table', class_='tinytable')

if table:
    # Extract table rows
    rows = table.find_all('tr')

    # Create an empty list to store the data
    data = []

    # Loop through rows and extract data
    for row in rows[1:]:  # Skip the header row
        cells = row.find_all('td')
        insider_info = {
            'Trade Date': cells[2].get_text(strip=True),
            'Value': cells[11].get_text(strip=True)
        }
        data.append(insider_info)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)

    # Remove dollar sign and comma from 'Value' column and convert to numeric type
    df['Value'] = df['Value'].str.replace('$', '').str.replace(',', '').astype(float)

    # Group the data by 'Trade Date' and aggregate the values
    df_merged = df.groupby('Trade Date').agg({'Value': 'sum'}).reset_index()

    # Save DataFrame to CSV
    df_merged.to_csv(f'data/{ticker}_insider_trades.csv', index=False)
    print('Insider trades data saved')

else:
    print('No table found')

Insider trades data saved


/var/folders/b7/3syr39bd24n_5m711ln6vz800000gn/T/ipykernel_69458/485494802.py:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Value'] = df['Value'].str.replace('$', '').str.replace(',', '').astype(float)


### Articles

Get articles

In [11]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

article_links = []

def get_yahoo_articles():
    driver.get(f"https://finance.yahoo.com/quote/{ticker}/news")

    # Scroll down the page multiple times to load more articles
    for _ in range(5): # scroll x times
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    page_source = driver.page_source
    driver.quit()

    soup = BeautifulSoup(page_source, 'html.parser')
    # stream_items = soup.find("ul", class_="stream-items x-large layoutCol1 svelte-1siuiba")
    
    articles = soup.find_all("li", class_="stream-item svelte-7rcxn")

    for article in articles:
        link = article.find("a")
        article_links.append(link.get("href"))

get_yahoo_articles()

print(article_links)

Get sentiment

In [5]:
def scrape_yahoo_finance(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get header
        header_element = soup.find(id="caas-lead-header-undefined")
        header = header_element.get_text().strip()

        # Get body
        article_elements = soup.find_all(class_="caas-body")
        body = ""
        for element in article_elements: body += element.get_text().strip() + "\n"

        # Get date
        date_text = soup.find_all(class_="caas-attr-time-style")[0].get_text().strip()
        date = re.search(r'(\w{3}, \w{3} \d{1,2}, \d{4})', date_text)
        date = datetime.strptime(date.group(), "%a, %b %d, %Y").strftime("%Y-%m-%d")
        return header, body, date
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

def analyze(text):
    # Summarizer
    summarizer_api_url = "https://api-inference.huggingface.co/models/sshleifer/distilbart-cnn-12-6"
    summarizer_headers = {f"Authorization": f"Bearer {config.huggingface_apikey}"}
    summarizer_output = requests.post(summarizer_api_url, headers=summarizer_headers, json={"inputs": text}).json()
    text = summarizer_output[0]['summary_text']

    # Sentimentizer
    sentimentizer_api_url = "https://api-inference.huggingface.co/models/ProsusAI/finbert"
    sentimentizer_headers = {"Authorization": f"Bearer {config.huggingface_apikey}"}
    sentimentizer_output = requests.post(sentimentizer_api_url, headers=sentimentizer_headers, json={"inputs": text}).json()

    most_likely_label, max_score = None, 0
    for sentiment in sentimentizer_output[0]:
        label, score = sentiment['label'], sentiment['score']
        if score > max_score: most_likely_label, max_score = label, score

    if most_likely_label == 'positive': sentiment = 1
    elif most_likely_label == 'neutral': sentiment = 0
    else: sentiment = -1

    return sentiment, summarizer_output, sentimentizer_output

# Main engine
articledata = []
company_table = pd.read_csv('companies.csv')
short_name, company_name = company_table.loc[company_table['ticker'] == ticker]['short name'].values[0], company_table.loc[company_table['ticker'] == ticker]['company name'].values[0]
for i, article in enumerate(article_links):
    try: 
        head, body, date = scrape_yahoo_finance(article)
        if ticker in head or short_name in head or company_name in head: pass
        else: continue

    except Exception as scrape_error:
        print(f"Failed to scrape article '{article}': {scrape_error.__class__.__name__}: {str(scrape_error)} \n")
        continue

    try: sentiment, summarizer_output, sentimentizer_output = analyze(body)
    except Exception as sentiment_error:
        print(f"Failed to analyze sentiment for article '{article}': {sentiment_error.__class__.__name__}: {str(sentiment_error)} \n")
        continue

    articledata.append({'Date': date, 'News_Sentiment': sentiment})
    print(f"Iteration {i}: {date}, '{head}'\n- Summarizer output:{summarizer_output[0]['summary_text']}\n- Sentimentizer output: {sentimentizer_output[0][0]['label']} ({round(sentimentizer_output[0][0]['score'], 2)})\n")

df = pd.DataFrame(articledata)
df_grouped = df.groupby('Date').sum().reset_index()
df_grouped.to_csv(f'data/{ticker}_sentiment.csv', index=False)

print("Success")

Salesforce Salesforce.com
Iteration 0: 2024-05-15, 'ServiceNow’s New Hiring Strategy: Poach From Salesforce'
- Summarizer output: ServiceNow Inc. has increased its workforce and expanded in enterprise software using a simple strategy: hire from its larger competitor, Salesforce . More of ServiceNow’s new employees have come from Salesforce than any other company . ServiceNow initially focused on tools to support information technology operations, but has moved into categories like customer service .
- Sentimentizer output: neutral (0.65)

Iteration 1: 2024-05-15, 'Salesforce Set to Exceed Q1 Estimates as Partners Report Healthy Deal Activity, BofA Says'
- Summarizer output: Salesforce Set to Exceed Q1 Estimates as Partners Report Healthy Deal Activity, BofA says . Salesforce is set to exceed Q1 estimates as partners report healthy deal activity . Bofa: Salesforce's Q1 sales will be higher than expected for Q1 quarter .
- Sentimentizer output: positive (0.96)

Iteration 4: 2024-05-13, '

# Merge

In [8]:
historical = pd.read_csv(f'data/{ticker}_daily.csv')
insider = pd.read_csv(f'data/{ticker}_insider_trades.csv')
news = pd.read_csv(f'data/{ticker}_sentiment.csv')

newdata = pd.merge(historical, insider, how='left', left_on='Date', right_on='Trade Date')
newdata = pd.merge(newdata, news, how='left', on='Date')

# newdata = newdata.drop(columns=['Open', 'High', 'Low', ])
newdata.rename(columns={'4. close': 'Close', '5. volume': 'Volume', 'Value': 'Insider_Trades'}, inplace=True)
newdata = newdata.drop(columns=['1. open', '2. high', '3. low', 'Trade Date'])

newdata.to_csv(f'data/{ticker}_merged.csv', index=False)

### Visualize